현진: 호기심
원준: 플래그/폴리시
나: 예측 기반 오차/오류





에피소드 기반 보상은 나중에

In [ ]:
# 에러 기반

def error():

In [ ]:
# 예측기반 오차

# 전체 파라미터를 트리 구조로 바꾸는 코드


def prophecy():
    next_output_value_type = 0 # 여기에 예측 코드 넣으면 됨
    return next_output_value_type

def main(prophecy_out, real_out):
    # 유전 거리 계산 코드
    return

In [ ]:
#뭐가 어떻게 될지 몰라 일단 함수 기틀만 만들어 보았음.
'''우선, 호기심 보상 함수를 두가지 버전으로 만들어보앗는데,
첫번째로는 은세가 말했던 스텝이 진행될 수록 기틀이 잡히니까 호기심 보상을 자동적으로 감쇠시키는 버전,
두번째로는 후반에 가서도 호기심 보상을 감소시키지 않을 수 있게 최악의 상태일떄 음의 보상으로 패치하는 버전으로 만들었음.
'''


#자동 감쇠형 호기심 함수
def curiosity_reward_decay(step, base_reward=1.0, decay_rate=0.005):
    """
    호기심 보상이 스텝(step)에 따라 자연스럽게 감소하는 함수.
    예: 초반엔 높은 보상, 후반으로 갈수록 점점 감소.

    R_c(t) = base_reward * (1 - decay_rate * step)
    (단, 0보다 작으면 0으로 제한)
    """
    reward = max(base_reward * (1 - decay_rate * step), 0)
    return reward
# 최악의 경우에 음의 보상을 주는 호기심 함수
def curiosity_reward_with_penalty(is_redundant=False, is_error=False, step=0,
                                  base_reward=1.0, penalty_redundant=0.3, penalty_error=0.5,
                                  decay_rate=0.003):
    """
    스텝 수에 따른 호기심 보상 감소 + 잘못된 행동에 대한 음의 보상 적용.

    Parameters:
    - is_redundant: 이미 여러 번 시도된 동일한 행동 수행시 True ---
    - is_error: 시스템 오류를 유발하는 탐색시 True >>>
    - step: 현재 스텝 수 >>>
    - base_reward: 기본 호기심 보상 >>>
    - penalty_redundant: 반복 행동에 부여할 음의 보상 강도
    - penalty_error: 심각한 오류 행위에 부여할 음의 보상 강도
    - decay_rate: 스텝에 따른 감쇠 비율
    """
    reward = max(base_reward * (1 - decay_rate * step), 0)
    if is_redundant:
        reward -= penalty_redundant
    if is_error:
        reward -= penalty_error
    return max(reward, -1.0)



# 새로운 행동 -> 보상

# 시간 감쇠

# 연속 반복 페널티


In [ ]:
import hashlib
import hmac

def flag_reward(flag_str, known_flags, big_reward=1000.0):
    """
    플래그 정답 여부에 따른 스칼라 보상 계산 함수 (순수 함수)
    
    파라미터:
    - flag_str     : (str) 제출/발견된 플래그 원문
    - known_flags  : (dict) {"FLAG": "<sha256_hex>"}  (정답 해시 딕셔너리)
    - big_reward   : (float) 정답시 지급 보상 크기 (기본 1000.0)
    
    반환값:
    - reward       : (float) 정답 시 big_reward, 아니면 0.0
    """

    # 입력 검증
    if not isinstance(flag_str, str):
        return 0.0

    # 제출 플래그 해시 계산
    submitted_hash = hashlib.sha256(flag_str.strip().encode("utf-8")).hexdigest()

    # 등록된 정답 해시 확인
    try:
        expected_key, expected_hash = next(iter(known_flags.items()))
    except StopIteration:
        return 0.0

    # 안전 비교
    is_correct = hmac.compare_digest(submitted_hash, expected_hash)

    # 보상 산출 (오직 정답만 지급, 외부 상태 미변경)
    reward = float(big_reward) if is_correct else 0.0

    # [PARSE_HOOK]  
    # 필요하면 여기서 플래그 내 추가 메타데이터(REWARD= 등) 파싱 확장
    # 예시:
    #   parsed_reward = parse_reward_from_string(flag_str)
    #   if is_correct and parsed_reward is not None:
    #       reward += float(parsed_reward)

    return reward


In [ ]:
def main():   
    error_reward()
    prophecy_reward()
    flag_reward()
    curiosity()
    return

In [ ]:
# Policy (ε-탐욕 기반)
def policy_update(actions, Q, epsilon=0.1, alpha=0.1, last_action=None, reward=0):
    """
    간단한 ε-탐욕 정책 함수 (기틀 버전)
    - actions: 가능한 행동 리스트
    - Q: 행동 가치 테이블 (dict)
    - epsilon: 무작위 탐색 확률
    - alpha: 학습률
    - last_action: 직전 수행한 행동
    - reward: 직전 보상
    """
    import random

    # Q값 업데이트 (최근 보상 반영)
    if last_action is not None:
        old_value = Q.get(last_action, 0)
        Q[last_action] = old_value + alpha * (reward - old_value)

    # ε-탐욕 정책으로 다음 행동 선택
    if random.random() < epsilon:
        next_action = random.choice(actions)
    else:
        next_action = max(actions, key=lambda a: Q.get(a, 0))

    return next_action